In [2]:
#If using Google Colab

from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [1]:
import torch 
print(torch.__version__)

1.3.0


In [1]:
import pandas as pd

df = pd.read_csv('data/text_emotion.csv')

In [2]:
df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [13]:
df['sentiment'].unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

In [14]:
len(df['sentiment'].unique())  #Number of different emotions

13

In [15]:
import numpy as np
import matplotlib.pyplot as plt

agg_list = np.array([len(i) for i in text])
avg_length = agg_list.mean()
max_length = np.array([len(i) for i in text]).max()
min_length = np.array([len(i) for i in text]).min()

print("The average length of the sequences before padding is: {} words".format(avg_length))
print("The max length of the sequences before padding is: {} words".format(max_length))
print("The min length of the sequences before padding is: {} words".format(min_length))

plt.hist(agg_list, bins = 20)
plt.show()

NameError: name 'text' is not defined

In [16]:
plt.figure(figsize=(12,8))
df['sentiment'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

<Figure size 864x576 with 0 Axes>

In [17]:
valid_emotion = ['neutral', 'worry', 'happiness', 'sadness', 'love']
df = df.loc[df['sentiment'].isin(valid_emotion)]

In [19]:
texts = df['content'].values
print(texts[0:5])

['Layin n bed with a headache  ughhhh...waitin on your call...'
 'Funeral ceremony...gloomy friday...'
 '@dannycastillo We want to trade with someone who has Houston tickets, but no one will.'
 "Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends"
 "I should be sleep, but im not! thinking about an old friend who I want. but he's married now. damn, &amp; he wants me 2! scandalous!"]


In [20]:
emotion_dict = {}
labels = df['sentiment']

for k in range(len(labels.unique())):
    emotion_dict[labels.unique()[k]] = k
    
print('Creating the emotion dict to encode the label')

Creating the emotion dict to encode the label


In [21]:
#Creating the encoded target

encoded_labels = np.array(list(labels.map(lambda x: emotion_dict[x])))

In [22]:
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.Defaults.create_tokenizer(nlp)

In [23]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer(preserve_case=False)
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [24]:
#Building a preprocessing loop over all of the rows
tokenized_text = []
vocab = []
for i in range(texts.shape[0]):
    text = texts[i]
    word_tokens = tokenizer.tokenize(text)
    filtered_sentence = [lemmatizer.lemmatize(w) for w in word_tokens if not w in stop_words]
    for word in filtered_sentence:
        if word not in vocab:
            vocab.append(word)
    tokenized_text.append(filtered_sentence)

In [25]:
word_id = {}
id_word = {}
for i, word in enumerate(vocab):
    word_id[word] = i
    id_word[i] = word

In [26]:
import os 

embeddings_index = {}
f = open(os.path.join("data/embedding/", 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [50]:
matrix_len = len(vocab) + 1
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0
emb_dim = 50

for i, word in enumerate(vocab):
    try: 
        weights_matrix[i] = embeddings_index[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))
        
weights_matrix[len(vocab)] = np.random.normal(scale=0.6, size=(emb_dim, )) #This is for the placeholder
print("Number of words found: {}".format(words_found))

Number of words found: 17180


In [51]:
pad_seq = 30

placeholder_token = len(vocab) # last token 

texts = np.full((len(tokenized_text), pad_seq), placeholder_token)
for i in range(len(tokenized_text)):
    sentence = tokenized_text[i]
    tokens = []
    if(len(sentence) < pad_seq):
        seq_lim = len(sentence)
    else:
        seq_lim = pad_seq
    for j in range(seq_lim):
        texts[i,j] = word_id[sentence[j]]

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(texts, encoded_labels, stratify=encoded_labels)

In [53]:
import os 

embeddings_index = {}
f = open(os.path.join("data/embedding/", 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [54]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 50))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

AttributeError: 'TweetTokenizer' object has no attribute 'word_index'

In [56]:
# First checking if GPU is available

import torch

train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [63]:
import torch.nn as nn


class Net(nn.Module):
    
    def __init__(self, num_layers, hidden_dim, 
                 embedding_dim, embedding_dict_size, 
                 seq_length, output_size):
        super(Net,self).__init__()
        
        self.n_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embedding_dict_size = embedding_dict_size
        self.seq_length = seq_length
        self.output_size = output_size
        
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(weights_matrix))
    
        #self.embedding = nn.Embedding(self.embedding_dict_size, self.embedding_dim)
        
        self.lstm = nn.LSTM(input_size=self.seq_length, hidden_size=self.hidden_dim, 
                            num_layers=self.n_layers, batch_first = True)
        
        self.dropout = nn.Dropout(0.3)
        
        self.fc1 = nn.Linear(self.hidden_dim, 24)
        self.fc2 = nn.Linear(24, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

        
    def forward(self, x, hidden):
        
        embeds = self.embedding(x)
        batch_size = embeds.shape[0]
        embeds = embeds.view(batch_size,self.embedding_dim,self.seq_length).type(torch.float)
        lstm_out, hidden = self.lstm(embeds)
        lstm_out = lstm_out.contiguous().view(batch_size, self.embedding_dim, self.hidden_dim)
        out1 = self.relu(self.fc1(lstm_out))
        out2 = self.softmax(self.fc2(out1))
        out2 = out2[:,-1,:]

        
        
        return out2, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [64]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
val_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

# dataloaders
batch_size = 64

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

In [65]:
len(emotion_dict)

5

In [66]:
#Parameter settting

from torch import optim

num_layers = 1
hidden_dim = 30
output_size = len(df['sentiment'].unique())
embedding_dim = 50
vocab_size = len(vocab) + 1
output_size = len(emotion_dict)

learning_rate = 0.1

net = Net(num_layers, hidden_dim, embedding_dim, vocab_size, pad_seq, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum=0.9)

In [67]:
print(net)

Net(
  (embedding): Embedding(39850, 50)
  (lstm): LSTM(30, 30, batch_first=True)
  (dropout): Dropout(p=0.3)
  (fc1): Linear(in_features=30, out_features=24, bias=True)
  (fc2): Linear(in_features=24, out_features=5, bias=True)
  (relu): ReLU()
  (softmax): Softmax()
)


In [68]:
from sklearn.metrics import f1_score, recall_score, balanced_accuracy_score

nb_epochs = 20
counter = 0
clip = 5
print_every = 64

if(train_on_gpu):
    net.cuda()

for epoch in range(nb_epochs):
    
    h = net.init_hidden(batch_size)
    net.train()
    
    for text_sequences, targets in train_loader:
        
        counter += 1
        
        #resetting the gradient
        net.zero_grad()
        
        if(train_on_gpu):
            text_sequences, targets = text_sequences.cuda(), targets.cuda()
            
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
            
        #Not sure why pytorch refuses the indices of text_sequences to be int.32
        input_text = text_sequences.type(torch.long)
        outputs, h = net(input_text, h) 
        loss = criterion(outputs, targets)
        loss.backward()
        
        #Clipping gradient to avoid exploding gradients
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        
        optimizer.step()
        
        
    # Get validation loss
    val_h = net.init_hidden(batch_size)
    val_losses = []
    y_pred_list = []
    y_true_list = []
    
    net.eval()
            
    for text_sequences, targets in val_loader:
        # Get validation loss

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        val_h = tuple([each.data for each in val_h])

        if(train_on_gpu):
            text_sequences, targets = text_sequences.cuda(), targets.cuda()
                
        input_text = text_sequences.type(torch.long)
        outputs, val_h = net(input_text, val_h)
        val_loss = criterion(outputs, targets)
        val_losses.append(val_loss.item())
                
        y_pred = outputs.cpu().detach().numpy().argmax(axis=1)
        y_pred_list += y_pred.tolist()
        y_true_list += targets.cpu().detach().numpy().tolist()
            
    net.train()
    print("Epoch: {}/{}  ".format(epoch+1, nb_epochs),
            "Step: {}  ".format(counter),
            "Loss: {:.6f}  ".format(loss.item()),
            "Val Loss: {:.6f}".format(np.mean(val_losses)),
            "Val accuracy: {:.6f}  ".format(balanced_accuracy_score(y_true_list, y_pred_list)),
            "Val F1: {:.6f}  ".format(f1_score(y_true_list, y_pred_list, average='weighted')),
            "Val recall: {:.6f}".format(recall_score(y_true_list,y_pred_list, average='weighted')))

Epoch: 1/20   Step: 367   Loss: 1.608528   Val Loss: 1.608900 Val accuracy: 0.210957   Val F1: 0.166541   Val recall: 0.290203


/Users/arnaudstiegler/anaconda3/envs/emotion_detection/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch: 2/20   Step: 734   Loss: 1.604962   Val Loss: 1.605748 Val accuracy: 0.200502   Val F1: 0.121741   Val recall: 0.276408
Epoch: 3/20   Step: 1101   Loss: 1.588000   Val Loss: 1.592223 Val accuracy: 0.200206   Val F1: 0.115465   Val recall: 0.270277
Epoch: 4/20   Step: 1468   Loss: 1.580858   Val Loss: 1.586906 Val accuracy: 0.201958   Val F1: 0.121401   Val recall: 0.272704
Epoch: 5/20   Step: 1835   Loss: 1.582125   Val Loss: 1.575706 Val accuracy: 0.199905   Val F1: 0.114966   Val recall: 0.270022
Epoch: 6/20   Step: 2202   Loss: 1.555011   Val Loss: 1.570532 Val accuracy: 0.200297   Val F1: 0.115988   Val recall: 0.270405
Epoch: 7/20   Step: 2569   Loss: 1.616198   Val Loss: 1.565045 Val accuracy: 0.222451   Val F1: 0.194838   Val recall: 0.301443
Epoch: 8/20   Step: 2936   Loss: 1.527717   Val Loss: 1.547754 Val accuracy: 0.231060   Val F1: 0.227589   Val recall: 0.312300
Epoch: 9/20   Step: 3303   Loss: 1.529103   Val Loss: 1.535990 Val accuracy: 0.239201   Val F1: 0.244498 

In [71]:
from sklearn.metrics import classification_report

print(classification_report(y_true_list, y_pred_list,target_names = list(emotion_dict.keys()),digits = 5))

              precision    recall  f1-score   support

     sadness    0.00000   0.00000   0.00000      1291
     neutral    0.34775   0.71111   0.46708      2160
       worry    0.28772   0.21277   0.24463      2115
        love    0.20000   0.00104   0.00207       961
   happiness    0.28260   0.39785   0.33046      1302

    accuracy                        0.31996      7829
   macro avg    0.22361   0.26455   0.20885      7829
weighted avg    0.24522   0.31996   0.25017      7829



In [48]:
val_h = net.init_hidden(1)
net('test',val_h)

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not str

In [85]:
model = torch.load('data/lstm_model.pt',map_location='cpu')

FileNotFoundError: [Errno 2] No such file or directory: 'data/lstm_model.pt'

In [ ]:
print(model)

In [18]:
model.eval()

Net(
  (embedding): Embedding(48998, 200)
  (lstm): LSTM(30, 50, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=50, out_features=13, bias=True)
)